phase1: preprocessing

In [1]:
#import library
import pandas as pd
import collections
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Download the punkt tokenizer if not already downloaded
nltk.download('punkt')
# Download the stopwords if not already downloaded
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ali\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ali\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# datafeame of Primary data
df = pd.read_csv("train_data.csv")
num_of_rows = len(df)
df.head(10)

,Unnamed: 0,textID,text,label,label_text
0,14951,7627d7ca56,Hi : Found you through . i am indeed a pigment...,1,neutral
1,6051,7527ed21ee,I`m so unhappy being here. I hate it. It`s aff...,0,negative
2,12712,2f6d9d1419,#stackeoverflow http://bit.ly/13Nfk1,1,neutral
3,2930,9afe83da6a,I`m completely exhausted thanks to yesterday. ...,0,negative
4,2174,322b61740c,"90 degrees, gross skies, and thunderstorms...p...",2,positive
5,8350,01e1f59fac,cant believe i stay out this late!.....waking ...,0,negative
6,10356,02fef838dd,_nobel ? ? ????????????? = ?????? ?????? ????...,1,neutral
7,4140,80c985a159,"Omg, my mom just called... Im too late... Hes ...",0,negative
8,18493,5a67377ff3,Engaged May 1st to the best guy I could not b...,2,positive
9,12923,0047d39e8b,Un mesaj de eroare: ForbiddenYou don`t have p...,1,neutral


In [3]:
# preprocess data into clean data
def preprocess(text):
    if isinstance(text, str):
        word_list = word_tokenize(text)
        stop_words = set(stopwords.words('english'))
        filtered_word_list = [word.lower() for word in word_list if word.lower() not in stop_words and len(word) > 2]
        
        return filtered_word_list
    
    return []

# iterate text columns from df and using preprocess function
total_words = list()
for i in range(num_of_rows):
    text_to_process = df["text"][i]
    result = list(preprocess(text_to_process))
    total_words.extend(result)

# Find the number of repetitions of words
count_data = collections.Counter(total_words)
unique_words = list(count_data.keys())
unique_words[0:10]




['found',
 'indeed',
 'pigment',
 'imagination',
 'unhappy',
 'hate',
 'affecting',
 'everything',
 'relationships',
 'people']

phase2: word count

In [4]:
# iterate text columns from df and using preprocess function
total_words = list()
positive_words = list()
negative_words = list()
neutral_words = list()

for i in range(num_of_rows):
    text_to_process = df["text"][i]
    result = list(preprocess(text_to_process))
    total_words.extend(result)
    
    #convert words into positive and negative and neutral categories
    if df.iloc[i]['label_text'] == 'positive':
        positive_words.extend(result)
        
    elif df.iloc[i]['label_text'] == 'negative':
        negative_words.extend(result)
        
    if df.iloc[i]['label_text'] == 'neutral':
        neutral_words.extend(result)

# Find the number of repetitions of words
count_data_total = collections.Counter(total_words)
count_data_positive = collections.Counter(positive_words)
count_data_negative = collections.Counter(negative_words)
count_data_neutral = collections.Counter(neutral_words)

#create dataframe with 4 columns words, positive , negative, neutral
words = count_data_total.keys()
pos = [count_data_positive[i] for i in words]
neg = [count_data_negative[i] for i in words]
neutral = [count_data_neutral[i] for i in words]

#show df
dict = {'word': words, 'positive': pos, 'negative': neg, 'neutral':neutral} 
df1 = pd.DataFrame(dict)

df2 = df1[df1['word'].isin(unique_words)]

# Drop rows where the sum is less than 10
row_sums = df1.sum(axis=1)
df2 = df1[row_sums >= 10]
df2.head(10)


C:\Users\Ali\AppData\Local\Temp\ipykernel_15184\3997826135.py:41: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  row_sums = df1.sum(axis=1)


,word,positive,negative,neutral
0,found,32,35,49
1,indeed,12,4,4
5,hate,5,193,46
7,everything,31,25,36
9,people,72,94,102
12,...,779,908,1061
14,http,301,171,413
16,completely,7,14,8
17,exhausted,1,13,6
18,thanks,402,18,63


phase3: caculating the probabilities of each word

In [5]:
df3 = df2.copy()

# Calculate the sum of each column to find the capacity of each class
sum_positive = df3['positive'].sum()
sum_negative = df3['negative'].sum()
sum_neutral = df3['neutral'].sum()
N_class = [sum_positive, sum_negative, sum_neutral] #[45074, 39758, 52176]

# Create a dictionary with 'word' as a separate key
temp = {'word': 'N_class', 'positive': sum_positive, 'negative': sum_negative, 'neutral': sum_neutral}
df3 = df3.append(temp, ignore_index=True)

# Calculate the probabilities
positive_probability = (df2['positive'] + 1) / (N_class[0] + len(df2))
negative_probability = (df2['negative'] + 1) / (N_class[1] + len(df2))
neutral_probability = (df2['neutral'] + 1) / (N_class[2] + len(df2))

df4 = pd.DataFrame()
df4["word"] = df3["word"].tolist()
df4["positive_probability"] = positive_probability
df4["negative_probability"] = negative_probability
df4["neutral_probability"] = neutral_probability

# Display the updated DataFrame
df4 = df4.dropna()
df4 = df4.reset_index(drop=True)
df4.head(10)

C:\Users\Ali\AppData\Local\Temp\ipykernel_15184\1126089810.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df3 = df3.append(temp, ignore_index=True)


,word,positive_probability,negative_probability,neutral_probability
0,found,0.000914,0.001156,0.001271
1,indeed,0.000360,0.000161,0.000127
2,...,0.000166,0.006229,0.001195
3,completely,0.000887,0.000835,0.000940
4,thanks,0.002022,0.003050,0.002618
5,yet,0.021608,0.029187,0.026994
6,perfect,0.008366,0.005523,0.010523
7,mood,0.000222,0.000482,0.000229
8,lol,0.000055,0.000450,0.000178
9,cant,0.011164,0.000610,0.001627


phase 4: get the log prior

In [6]:
#normalize
df5 = df4.copy()
# Normalize each row with
df5.iloc[:, 1:] = df5.iloc[:, 1:].div(df5.iloc[:, 1:].sum(axis=1), axis=0)
# Display DataFrame

# Save the dataframe to a CSV file
df5.to_csv('train_result.csv', index=False)
df5.head(10)

,word,positive_probability,negative_probability,neutral_probability
0,found,0.273629,0.345977,0.380394
1,indeed,0.555965,0.247840,0.196195
2,...,0.021900,0.820702,0.157398
3,completely,0.333045,0.313634,0.353321
4,thanks,0.262956,0.396626,0.340418
5,yet,0.277781,0.375205,0.347014
6,perfect,0.342712,0.226229,0.431060
7,mood,0.237790,0.516762,0.245448
8,lol,0.081139,0.658299,0.260562
9,cant,0.833087,0.045524,0.121389
